In [ ]:
import cv2 

cap = cv2.VideoCapture("rtsp://wowzaec2demo.streamlock.net/vod/mp4:BigBuckBunny_115k.mov")

while True:
    ret, frame = cap.read()

    if not ret:
        print("Error reading frame. Check the RTSP stream.")
        break

    cv2.imshow('RTSP Stream', frame)
    
    if cv2.waitKey(1) == 13:  # 13 is the Enter Key
        break

# Release camera and close windows
cap.release()
cv2.destroyAllWindows()


In [2]:
!pip install imagezmq


In [ ]:
# run this program on the Mac to display image streams from multiple RPis
import cv2
import imagezmq

image_hub = imagezmq.ImageHub()

while True:  # show streamed images until Ctrl-C
    rpi_name, image = image_hub.recv_image()
    cv2.imshow(rpi_name, image) # 1 window for each RPi
    cv2.waitKey(1)
    image_hub.send_reply(b'OK')

In [ ]:
# autoconnect 

In [ ]:
import cv2
import requests  
import time  

class VideoCapture:
    def __init__(self, cam_address, cam_force_address=None, blocking=False):
        """
        cam_address: ip address of the camera feed
        cam_force_address: ip address to disconnect other clients (forcefully take over)
        blocking: if true read() and connect_camera() methods blocked until ip camera is reconnected
        """
        self.cam_address = cam_address
        self.cam_force_address = cam_force_address
        self.blocking = blocking
        self.capture = None
        
        # NOTE: Can be increased to reduce printing
        self.RECONNECTION_PERIOD = 0.5
        # Calls the connect method
        self.connect_camera()

    def connect_camera(self):
        print("Connecting...")
        while True:
            try:
                if self.cam_force_address is not None:
                    requests.get(self.cam_force_address)

                self.capture = cv2.VideoCapture(self.cam_address)

                if not self.capture.isOpened():
                    time.sleep(self.RECONNECTION_PERIOD)
                    raise Exception("Could not connect to a camera: {0}".format(self.cam_address))

                print("Connected to a camera: {}".format(self.cam_address))

                break
            except Exception as e:
                print(e)

                if self.blocking is False:
                    break

                time.sleep(self.RECONNECTION_PERIOD)

    def getStream(self):
        """
        Reads frame and if frame is not received tries to reconnect the camera

        :return: ret - bool witch specifies if frame was read successfully
                 frame - opencv image from the camera
        """

        ret, frame = self.capture.read()

        # If feed goes down we try to reconnect
        if ret is False:
            self.connect_camera()

        return ret, frame

In [ ]:
cap = VideoCapture("rtsp://wowzaec2demo.streamlock.net/vod/mp4:BigBuckBunny_115k.mov")

while(1):
    ret, frame = cap.getStream()
    
    # Note this will keep the loop running until you force the program to exit
    try:
        cv2.imshow('RTSP Stream', frame)
    except:
        print("Feed has gone down...")
        
    if cv2.waitKey(1) == 13: #13 is the Enter Key
        print("Exited...")
        break
        
# Release camera and close windows
cv2.destroyAllWindows()   